In [ ]:
!wget https://raw.githubusercontent.com/SciX-UNSW/D2D/main/transit-spectrum-W39b-G395H-10pix_weighted-average.nc

# Function to plot the JWST WASP-39b spectrum
def JWST_frequecies_intensities2():
    # Reading weighted average spectral data
    file_strings = ['transit-spectrum-W39b-G395H-10pix_weighted-average.nc']
    spec_names = ['average']

    data_check = xr.open_dataset('transit-spectrum-W39b-G395H-10pix_weighted-average.nc')

    nbins = len(data_check['central_wavelength'].values[5:])
    nspec = len(file_strings)
    gauss_x = np.linspace(-0.001, 0.001, 1000)

    # Creating arrays of 1x339 dimensions
    # -----------------------------------

    # Wavelengths (+ error)
    w_array = np.empty([nspec,nbins])
    we_array = np.empty([nspec,nbins])

    # Transition depth (+ error)
    d_array = np.empty([nspec,nbins])
    de_array = np.empty([nspec,nbins])

    dr_array = np.empty([nspec,nbins])
    dr_mean = np.empty([nspec])
    dr_std = np.empty([nspec])

    pdf_norm = np.empty([nspec,len(gauss_x)])

    align_d_array = np.empty([nspec,nbins])

    for i in range(0, nspec):
        data = xr.open_dataset(file_strings[i])
        data_keys = list(data.keys())

        # Collecting data (wavelength, wavelength error, and transit depth) from weighted average spectrum file
        w = data['central_wavelength'].values[5:]
        we = data['bin_half_width'].values[5:]
        d = data['transit_depth'].values[5:]

        # Collecting errors from transit depth
        if (data_keys[1]=='transit_depth_error_pos'):
            de = data['transit_depth_error_pos'].values[5:]
        else:
            de = data['transit_depth_error'].values[5:]

        # Moving data from the weighted average spectrum file to their corresponding arrays created before
        data_sort = np.argsort(w)
        w_array[i,:] = w[data_sort]
        d_array[i,:] = d[data_sort]
        de_array[i,:] = de[data_sort]
        we_array[i,:] = we[data_sort]

    weights_de = 1./de_array**2
    average_d, sum_of_weights = np.average(d_array, weights=weights_de, axis=0, returned=True)
    median_d = np.median(d_array, axis=0)

    weighted_average_de = np.sqrt(1./ np.sum(weights_de, axis=0))
    average_de = np.mean(de_array, axis=0)
    median_de = np.median(de_array, axis=0)

    std_array = np.std(d_array, axis=0)

    for i in range(0, nspec):

        dr_array[i,:] = d_array[i,:] - average_d

        dr_mean[i] = np.mean(dr_array[i,:])
        dr_std[i] = np.std(dr_array[i,:])

        pdf_norm[i,:] = norm.pdf(gauss_x, loc=dr_mean[i], scale=dr_std[i])


    # aligned average
    for i in range(0, nspec):
        align_d_array[i,:] = d_array[i,:] - dr_mean[i]

    align_average_d = np.average(align_d_array, weights=weights_de, axis=0)

    data_wavelength = w_array[0,:]
    data_wavelength_err = we_array[0,:]

    JWSTData = pd.DataFrame()
    JWSTData['Wavelength'] = data_wavelength
    JWSTData['Wavelength_Err'] = data_wavelength_err

    JWSTData['Wavenumber'] = tick_function(data_wavelength)
    JWSTData['Wavenumber'] = pd.to_numeric(JWSTData['Wavenumber'])

    JWSTData['TransDepth'] = average_d
    JWSTData['TransDepth_Err'] = average_de

    scale = 1.e2

    JWSTData['TransDepth'] = JWSTData['TransDepth']*scale
    JWSTData['TransDepth_Err'] = JWSTData['TransDepth_Err']*scale

    freq = np.array(JWSTData['Wavenumber'])
    intens = np.array(JWSTData['TransDepth'])#/np.nanmax(JWSTData[['TransDepth']].values)
    err = np.array(JWSTData['TransDepth_Err'])
    return freq, intens, err
    
freq, intens, err = JWST_frequecies_intensities2()
df = pd.DataFrame({
    'Frequency': freq,
    'Intensity': intens,
    'Error': err
	})

# Export the DataFrame to a CSV file
df.to_csv("combined_data.csv", index=False)

#---------------------------------

# Function to convert from cm-1 to um
def wavenumbers2microns(x):
    return 10000/x

# Function to convert from um to cm-1
def microns2wavenumbers(x):
    return 10000/x

# Function to change from cm-1 to um
def tick_function(X):
    V = 1.0000000e4/(X)
    return ["%.2f" % z for z in V]

# Function to normalise vibrational intensities
def norminten(tbnorm):
    norm = []
    upp = max(tbnorm)
    for entry in tbnorm:
        norm.append(entry/upp)
    return norm